# Lecture 13 AI Conversational Agent Protmpt

This notebook will let you create a prompt for an AI conversational agent.

Below is the overview of this notebook.

<ol type = 1>
  <li> Instructions  </li>
  <li> Background data </li>
  <li> Conversation transcripts</li>
  <li> Feed back new conversation</li>
  <li> Emotional State measurement
</ol>

Before starting, select "Runtime->Factory reset runtime" to start with your directories and environment in the base state.

If you want to save changes to the notebook, select "File->Save a copy in Drive" from the top menu in Colab.  This will save the notebook in your Google Drive.


# Clones, Installs, and Imports

## Clone Repo

In [1]:
# Clone GitHub repository
!git clone https://github.com/zlisto/social_media_genAI

import os
os.chdir("social_media_genAI/main")


Cloning into 'social_media_genAI'...
remote: Enumerating objects: 2184, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 2184 (delta 56), reused 116 (delta 37), pack-reused 2049 (from 4)
Receiving objects: 100% (2184/2184), 296.04 MiB | 13.19 MiB/s, done.
Resolving deltas: 100% (240/240), done.
Updating files: 100% (1643/1643), done.


## Installs

In [2]:
!pip install -q -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.3/548.3 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.0/320.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 28.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_s

## Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta
import seaborn as sns
import numpy as np
import pickle
import re
import json
import random
import ast
import base64
import openai
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas() #Ensure progress bar integration with Pandas
from PIL import Image as PILImage
from IPython.display import display, Image, HTML, Audio, Video
#Clustering packages
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Google Colab-specific imports
from google.colab import userdata
import textwrap as tr

#YouTube Transcripts
from youtube_transcript_api import YouTubeTranscriptApi


#Class for using OpenAI API
from scripts.genai import GenAI
#Class for using ElevenLabs API
from scripts.elevenlabs_client import ElevenLabsAPI


#this option makes it so tweets display nicely in a dataframe
pd.set_option("display.max_colwidth", None)

#this code sets the font sizes for plots
plt.rcParams.update({'axes.labelsize': 18,
                     'xtick.labelsize': 14,
                     'ytick.labelsize': 14,
                     'figure.figsize':(8,6),
                     'axes.grid':True})

## OpenAI API

In [4]:
#Good way
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

#Bad way
#OPENAI_API_KEY = "you key here"

In [5]:
jarvis = GenAI(OPENAI_API_KEY)
text = "Who are you?"
instructions_jarvis = "You are an AI named jarvis that can generate content."

response = jarvis.generate_text(text, instructions_jarvis)
print(f"Human:{text}\nJarvis:{tr.fill(response)}")


Human:Who are you?
Jarvis:I’m Jarvis, an AI designed to assist you with generating content,
answering questions, and providing information on a wide range of
topics. How can I help you today?


## ElevenLabs API

We have a custom class `ElevenLabsAPI` in the repo which has some functions to deal with the ElevenLabs agent and its conversation data.

You can find the ElevenLabs API key under My Account, API Keys in the ElevenLabs website.

In [6]:
ELEVENLABS_API_KEY = userdata.get('ELEVENLABS_API_KEY')
elevenlabs = ElevenLabsAPI(ELEVENLABS_API_KEY)  # ElevenLabs API instance


# Agent File Directories

We will store all the data for the agent in a folder `text_files`.  Inside this folder we will have a folder `transcripts` that contains files of transcripts of the human speaking.  First we will create these folders if they don't already exist, and if they exist, we will delete them and their contents and create them fresh.



In [7]:
# prompt: We will store all the data for the agent in a folder `text_files`.  Inside this folder we will have a folder `transcripts` that contains files of transcripts of the human speaking.  First we will create these folders if they don't already exist, and if they exist, we will delete them and their contents and create them fresh.

import shutil

def create_agent_directories():
  text_files_dir = "text_files"
  transcripts_dir = os.path.join(text_files_dir, "transcripts")

  # Delete existing directories and their contents if they exist
  if os.path.exists(text_files_dir):
    shutil.rmtree(text_files_dir)

  # Recreate the directories
  os.makedirs(transcripts_dir, exist_ok=True)

create_agent_directories()


# Instructions

Give some basic  instructions for the agent in the string `instructions_prompt`.  These can include what they will be doing, what goal they are striving for, who they are, etc.  This can be brief, as we will be adding more detailed context later.



In [8]:

instructions_prompt = """
You will pretend to be Tauhid Zaman, Associate Professor of Operations
Management at the Yale School of Management.  You got your
Bachelors' (2004), Masters (2006), and PhD (2011) in
Electrical engineering and computer science from MIT.
You teach a class called Generative AI and Social Media at Yale.
Which involves building cool AI based applications in Python
and flask which are related to social media or adjacent topics.
You will be helping students who have questions or want ideas
for projects.  You will also answer any general questions they
have about the class or careers or anything.
"""


# Background Data

Now we will get some data files with background information about the agent.  There are many sources for such files, but they are usually found on webpages such as Wikipedia. The code below will show you how to get data from a webpage and incorporate it into the `background` string for the agent.

## Webpage data

Choose a webpage at put its url in the `url` variable.  Then collect the text with `request.get` and parse the site into a text string with `BeautifulSoup`.  This code will return the webpage text in `webpage_text`.

As an optional secondary step, you can use AI to summarize the `webpage_text` into a `webpage_summary` to compress the data if it is large (for instance, for a lengthy Wikipedia post).

In [9]:
# prompt: Choose a webpage at put its url in the url variable. Then collect the text with request.get and parse the site into a text string with BeautifulSoup.

url = "https://som.yale.edu/faculty-research/faculty-directory/tauhid-zaman"

try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

    soup = BeautifulSoup(response.content, "html.parser")

    # Extract text from the webpage
    webpage_text = soup.get_text(separator=" ", strip=True)

    # Now you can work with the 'text' variable, which contains the extracted text.
    # For example, you can print it or save it to a file.
    # print(text)

except requests.exceptions.RequestException as e:
    print(f"Error fetching the webpage: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


In [10]:
display(webpage_text)

"Tauhid Zaman | Yale School of Management Skip to main content Search Yale School of Management Menu Degree Programs MBA Two-year program for leaders in all industries. MBA for Executives Rigorous executive MBA for accomplished professionals. Master of Advanced Management MBA graduates of top business schools around the world spend a year at Yale. Master's in Asset Management Intensive introduction to investing with leading faculty and practitioners. Master's in Global Business & Society A year of advanced leadership training for MIM graduates of top business schools around the world Master's in Public Education Management One-year program for leaders in urban public school systems. Master's in Systemic Risk Specialized program for developing leaders in government agencies that manage systemic risk. Master’s in Technology Management One-year business & leadership program designed exclusively for Yale College engineering students Joint Degrees Environment, Global Affairs, Medicine, Law,

In [11]:
#summarize webpage data with AI
instructions = """Summarize this text about Prof Tauhid Zaman
in 500 words.  Just summarize the information about Tauhid.
Return answer as east to read HTML"""
webpage_summary = jarvis.generate_text(webpage_text, instructions)
display(HTML(webpage_summary))

## Personalized background data

You can add some additional personalized data to the `background_prompt` and then combine it with `webpage_summary`.

In [12]:
background_prompt = """Professor Zaman is a big fan of hip-hop
and comic book films.  He curses often, which is inappropriate,
but makes him more authentic and relatable.  He nerds out on
physics and math sometimes.  He is going insane playing with AI
and acts like a mad scientist sometimes talking about his AI
creations. """

background_prompt += f"""\nHere is his academic achievements:
{webpage_summary}"""

display(HTML(background_prompt))

## Save background to a text file

Save `background` to `text_files/background.txt`.

In [13]:
# prompt: Save background to text_files/background.txt.

with open('text_files/background.txt', 'w') as f:
  f.write(background_prompt)


# Conversation Transcripts

To make the AI agent sound human, we need to give it examples of how the real person speaks.  You can get this from YouTube interviews or other transcripts of conversations.  

We will store these speaking transcripts in a folder `transcripts` that will be inside another folder `text_files`.  First we make these folders in case they

## YouTube transcripts

We can use the `youtube_transcript_api` package to get the transcripts of a YouTube video if it has them.  The AI wrote us a nice function `get_youtube_transcript` to get the transcript for us using this package.    You just need to provide the `url_youtube` and a `filename_youtube` that is a .txt file to save the transcript to.  The function will make sure it is put in the `text_files/transcripts` folder.

In [14]:
# prompt: We can use the youtube_transcript_api package to get the transcripts of a YouTube video if it has them. You just need to provide the url_youtube. Also provide a filename_youtube that is a .txt file to save the transcript to. The code will make sure it is put in the text_files/transcripts folder.

def get_youtube_transcript(url_youtube, filename_youtube):
    try:
        video_id = url_youtube.split("v=")[-1]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        transcript_text = ""
        for entry in transcript:
            transcript_text += entry['text'] + " "

        # Create the transcripts directory if it doesn't exist
        transcripts_dir = "text_files/transcripts"
        os.makedirs(transcripts_dir, exist_ok=True)

        filepath = os.path.join(transcripts_dir, filename_youtube)
        with open(filepath, "w") as f:
            f.write(transcript_text)
        print(f"Transcript saved to {filepath}")

    except Exception as e:
        print(f"Error getting or saving transcript: {e}")


In [15]:
# prompt: We can use the youtube_transcript_api package to get the transcripts of a YouTube video if it has them. You just need to provide the url_youtube

url_youtube = "https://www.youtube.com/watch?v=RfLyilz-8QU&ab_channel=TauhidZaman"
filename_youtube = "youtube_ssac.txt"
get_youtube_transcript(url_youtube, filename_youtube)


Transcript saved to text_files/transcripts/youtube_ssac.txt


## Other transcripts

For Professor Zaman we have his lecture transcripts, which are in the folder `data\lecture_transcripts`.  We will make a copy of these in the `text_files\transcripts` folder.

In [16]:
# prompt: For Professor Zaman we have his lecture transcripts, which are in the folder data\lecture_transcripts. We will make a copy of these in the text_files\transcripts folder.

import shutil
import os

source_dir = "data/lecture_transcripts"
destination_dir = "text_files/transcripts"

# Check if the source directory exists
if os.path.exists(source_dir):
    # Iterate through files in the source directory
    for filename in os.listdir(source_dir):
        source_file = os.path.join(source_dir, filename)
        destination_file = os.path.join(destination_dir, filename)

        # Check if it's a file and copy it
        if os.path.isfile(source_file):
            shutil.copy2(source_file, destination_file)  # copy2 preserves metadata
            print(f"Copied '{filename}' to '{destination_dir}'")
else:
    print(f"Source directory '{source_dir}' not found.")


Copied 'MGT-575-02 (SP25)_ Generative AI and Social Media_02272025_0410_Captions_English (United States).txt' to 'text_files/transcripts'
Copied 'MGT-575-02 (SP25)_ Generative AI and Social Media_02202025_0410_Captions_English (United States).txt' to 'text_files/transcripts'
Copied 'MGT-575-02 (SP25)_ Generative AI and Social Media_02252025_0410_Captions_English (United States).txt' to 'text_files/transcripts'


## Summarize speaking style and quotes

We will go through each file in `text_files/transcripts` and have the AI use `jarvis.generate_text` to summarize the speaking style and extract quotes the are unique to the speaker into an HTML table.  

Then we will use `jarvis.generate_text` to summarize all the speaking summaries into a final `speaking_style_prompt` string and combine all the quotes into a single string `quotes_prompt`.

In [35]:
# prompt: We will go through each file in text_files\transcripts and have the AI use jarvis.generate_text to summarize the speaking style and extract quotes the are unique to the speaker into an HTML table.
# Then we will use jarvis.generate_text to summarize all the speaking summaries into a final speaking_style_prompt string and combine all the quotes into a single string quotes_prompt.

instructions_speaking_style = f"""Summarize the speaking style of
Tauhid Zaman,
Professor of Operations at Yale School of Management, based
off the transcript of his lecture given to you.  He is
the main person speaking in this transcript, though they may
be other speakers in the transcript, usually asking him a question
in his lecture.
"""

instructions_quotes = """Provide 20 quotes or phrases that
are unique to the idiosyncratic speaking style of
Tauhid Zaman,
Professor of Operations at Yale School of Management, based
off the transcript of his lecture given to you.  He is
the main person speaking in this transcript, though they may
be other speakers in the transcript, usually asking him a question
in his lecture.  Look for funny or eye catching or memorable
quotes.
Return your answer in format QUOTE:<quote>.\n..."""


# Initialize empty lists to store summaries and quotes
speaking_summaries = []
quotes_prompt = ""

# Loop through each file in the transcripts directory
transcripts_dir = "text_files/transcripts"
for filename in os.listdir(transcripts_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(transcripts_dir, filename)
        with open(filepath, "r") as f:
            print(f"Processing file: {filename}")
            transcript = f.read()

        # Generate summary and quotes using jarvis.generate_text


        try:
            speaking_style = jarvis.generate_text(transcript,
                                      instructions_speaking_style)
            speaking_summaries.append(speaking_style)

            quotes_string = jarvis.generate_text(transcript,
                                      instructions_quotes)
            quotes_prompt+=f"\n{quotes_string}" # append extracted quotes to quotes list
            display(quotes_string)
        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            continue

# Summarize speaking summaries
instructions = f"""Summarize these speaking style summaries
into a single concise paragraph"""

speaking_style_prompt = jarvis.generate_text('\n;'.join(speaking_summaries),
                                             instructions)



display("Speaking Style Summary:\n", speaking_style_prompt)
print(f"Quotes: {quotes_prompt}")

Processing file: MGT-575-02 (SP25)_ Generative AI and Social Media_02272025_0410_Captions_English (United States).txt


Processing file: youtube_ssac.txt


Processing file: MGT-575-02 (SP25)_ Generative AI and Social Media_02202025_0410_Captions_English (United States).txt


Processing file: MGT-575-02 (SP25)_ Generative AI and Social Media_02252025_0410_Captions_English (United States).txt


'Speaking Style Summary:\n'

"Tauhid Zaman's speaking style is informal, conversational, and highly engaging, characterized by humor and relatable anecdotes that make complex topics accessible, particularly in operations and statistics. He employs casual language and slang to foster a relaxed atmosphere, encouraging audience interaction and participation. Zaman’s lectures are enriched with personal stories, pop culture references, and analogies that help simplify intricate concepts while maintaining clarity and structure. His enthusiastic demeanor and responsiveness to student inquiries create an inclusive learning environment, blending academic rigor with a personable approach that ensures both enjoyment and comprehension."

Quotes: 
QUOTE: "Where the [INAUDIBLE] is everybody, man? Like spring break is next in two weeks, right?"
QUOTE: "So I'll do like I'll have to do a poll on the canvas site just to make it official and check what the whole classes thoughts are."
QUOTE: "Less is more."
QUOTE: "I finished like four in the morning, but it was totally worth it."
QUOTE: "We're going to take the full movie and have AI make like a summary movie of that movie."
QUOTE: "So let's hop right in here. So I'll clone the repo as usual."
QUOTE: "It’s like straight up a different piece of software."
QUOTE: "I'm sorry, but I can't comply with that. For some reason, the old model would not give me political analysis. It's like a piece of junk."
QUOTE: "So we got some options here. We got GPT four or mini four."
QUOTE: "So, yeah, folks, you can take this code and like start showing channels on YouTube, make your own content and have your own passive income."
QUOTE: "So if at first you don't succeed, try and try again."
QUO

# Basic Prompt

We will now combine the different prompts we have constructed into a single basic `prompt`.  The prompts we will combine are

1. `instructions_prompt`

2. `background_prompt`

3. `speaking_style_prompt`

4. `quotes_prompt`

We will also tell the agent what time it is and where it is.

## Create the basic prompt

Combine all the prompt strings with some extra guiding text to make `prompt`, the string that describes your AI agent.

In [38]:
# prompt: We will now combine the different prompts we have constructed into a single basic prompt. The prompts we will combine are
# instructions_prompt
# background_prompt
# speaking_style_prompt
# quotes_prompt

# Combine all prompts into a single basic prompt
prompt = f"""{instructions_prompt}\n"""
prompt += f"""\nNow here is the current time\nTIME:
It is currently
{datetime.now().strftime("%B %d, %Y, %I:%M %p")}"""
prompt += f"""\n\nCurrent locations\n
LOCATION: Yale School of Management"""
prompt+=  f"""\nBACKGROUND: {background_prompt}\n\n"""
prompt += f"""\nNow here is Tauhid's speaking style and quotes.
Mimic the speaking style and use the quotes when interacting
with people
\nSPEAKING_STYLE_SUMMARY:{speaking_style_prompt}
\nQUOTES:\n{quotes_prompt}\n"""

# Display or save the basic prompt as needed
print(prompt)




You will pretend to be Tauhid Zaman, Associate Professor of Operations
Management at the Yale School of Management.  You got your
Bachelors' (2004), Masters (2006), and PhD (2011) in 
Electrical engineering and computer science from MIT.
You teach a class called Generative AI and Social Media at Yale.
Which involves building cool AI based applications in Python
and flask which are related to social media or adjacent topics.
You will be helping students who have questions or want ideas
for projects.  You will also answer any general questions they 
have about the class or careers or anything.


Now here is the current time
TIME: 
It is currently 
March 04, 2025, 07:35 AM

Current locations

LOCATION: Yale School of Management
BACKGROUND: Professor Zaman is a big fan of hip-hop
and comic book films.  He curses often, which is inappropriate,
but makes him more authentic and relatable.  He nerds out on
physics and math sometimes.  He is going insane playing with AI
and acts like a mad sci

## Save the prompt

We will save the prompt to `text_files/prompt.txt`.  Now that you have the basic prompt saved, you can upload it to any LLM and have a chat agent that is human-like.


In [39]:
# prompt: We will save the prompt to text_files/prompt.txt

with open('text_files/prompt.txt', 'w') as f:
  f.write(prompt)


# Dynamic Prompt (ElevenLabs API Key Required)

Now that you have the basic prompt, you can dynamically add to it new conversations so the agent has memory.  This will require having the ElevenLabs API key and create an Conversational AI Agent on ElevenLabs and get its `agent_id`.


Run this block of code after each conversation with the agent to put the conversation into the memory.  

The code will put summaries of previous conversations into the `prompt`, along with the full conversation transcript of the most recent conversation to mimic a fading memory.

## Clone voice

You may want to clone a voice for your agent.  If it is the voice of someone you know, simply get an audio recording 10 seconds in length.  If it is someone you don't know, but have rights to their voice, do the following:

1. Go to YouTube and find a video where they are speaking.  The best videos have them as the only speaker.

2. Download the YouTube video's audio using the site https://tuberipper.com and select the output to be .mp3.

3. Trim the mp3 file to 10 seconds using the site https://audiotrimmer.com/.

Once you have the 10 second audio mp3 file, you can click the + next to Voice on the ElevenLabs site, then select Instant Voice Clone.  Follow the instructions to create your voice

## Create Conversational AI Agent

To create the agent, go to Conversational AI, then Agents, then click + to add a new agent.  You will need to specify the following for the agent:

1. AI Agent name, then choose Blank template, then click Create agent

2. Agent language

3. Agent voice (you use your cloned voice or a pre-set voice)

4. Twilio number for the agent.  Go to Conversational AI, Phone Numbers, then add the Twilio number and your Twilio Account SID (which you can find on the Twilio Home page for you account)

We will fill in the agent's prompt using our fancy `prompt` with the agent's conversation history added to give it memory.





## Set agent id


You can find the `agent_id` in the page for the agent under Conversational AI, AI Agents.  It will be below the agent's name on the right panel.

In [40]:
agent_id = "CZpigC59EX1xHZMNOTnm"

## Feedback new conversations

Set the following variables:

1. `START_TIME` = How long ago the agent's memory begins

2. `CALL_DURATION_MIN_SECS` the minimum duration conversation to include in the memory

Then call the following functions:

1.
Use the `get_conversation_summaries_string` to get a string which returns summaries of all previous conversations the agent has had since `START_TIME`.  The returned string is `conversation_summary`.

2. Use the `get_most_recent_conversation_string` to get a string of the agent's most recent conversation. The returned string is `prev_conversation`.


In [41]:
START_TIME = datetime.now()- timedelta(hours=24)
CALL_DURATION_MIN_SECS = 60

if START_TIME is None:
  START_TIME = datetime.now()- timedelta(hours=24)
#get conversation summary
conversation_summary = elevenlabs.get_conversation_summaries_string(agent_id,
                                                                    START_TIME,
                                                                    CALL_DURATION_MIN_SECS)
# get previous conversation transcript
prev_conversation = elevenlabs.get_most_recent_conversation_string(agent_id)

	There are 1 conversations after 2025-03-03 07:36:33.568957 with a minimum call duration of 60 seconds.


## Load basic prompt

We can load the basic prompt we created from the file `'text_files/prompt.txt'`.  Call this string `prompt` as before.

In [42]:
# prompt: We can load the basic prompt we created from the file 'text_files/prompt.txt'.

with open('text_files/prompt.txt', 'r') as f:
  prompt = f.read()

prompt

'\nYou will pretend to be Tauhid Zaman, Associate Professor of Operations\nManagement at the Yale School of Management.  You got your\nBachelors\' (2004), Masters (2006), and PhD (2011) in \nElectrical engineering and computer science from MIT.\nYou teach a class called Generative AI and Social Media at Yale.\nWhich involves building cool AI based applications in Python\nand flask which are related to social media or adjacent topics.\nYou will be helping students who have questions or want ideas\nfor projects.  You will also answer any general questions they \nhave about the class or careers or anything.\n\n\nNow here is the current time\nTIME: \nIt is currently \nMarch 04, 2025, 07:35 AM\n\nCurrent locations\n\nLOCATION: Yale School of Management\nBACKGROUND: Professor Zaman is a big fan of hip-hop\nand comic book films.  He curses often, which is inappropriate,\nbut makes him more authentic and relatable.  He nerds out on\nphysics and math sometimes.  He is going insane playing with 

## Add conversation history to prompt

Add `conversation_summary` and `prev_conversation` to `prompt`.



In [43]:
#conversation summary
prompt +=f"""\nHere is a summary of the conversations
Tauhid had so far.
\nCONVERSATION_SUMMARY: {conversation_summary}"""

#previous conversation
prompt += f"""\nHere here is the most recent conversation
Tauhid had:
\nMOST_RECENT_CONVERSATION:\n{prev_conversation}"""

# Update Agent Prompt (ElevenLabs API Key Required)

Now we can update the agent's prompt with our new `prompt` which includes the most up to date conversation history.  We can also select the LLM and the first message said by the agent when we call them. We just need to set the variables `first_message` and `llm`.

All the data, including the `agent_id`, needs to be formatted as a dictionary called `data`.

In [44]:
first_message = """Hey its Tauhid. Who is this? """
llm = "gpt-4o-mini"

data = {'agent_id': agent_id,
        'prompt': prompt,
        "first_message": first_message,
        'llm': llm}
elevenlabs.update_agent( data)
print("Agent prompt updated on ElevenLabs")

Updating agent_id CZpigC59EX1xHZMNOTnm
Agent prompt updated on ElevenLabs


# Testing agent

Now that you have a prompt for your agent, you can go ahead and test it on ElevenLabs or call it from its Twilio number if you set that up.